In [1]:
from tpot import TPOTClassifier
import numpy as np
from sklearn.model_selection import GroupShuffleSplit

In [ ]:
fmat_file = '../ppi_ml/data/featmats/featmat_final_labeled.pkl'

In [ ]:
pipeline_optimizer = TPOTClassifier()

In [ ]:
pipeline_optimizer = TPOTClassifier(generations=5, population_size=20, cv=5,
                                    random_state=42, verbosity=2)

`X_train` = array of training data, `y_train` = labels, `group` = protein cmplx group

In [ ]:
gss = GroupShuffleSplit(n_splits=2, train_size=.7, random_state=13)
gss.get_n_splits()

In [ ]:
for i, (train_index, test_index) in enumerate(gss.split(X, y, groups)):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}, group={groups[train_index]}")
    print(f"  Test:  index={test_index}, group={groups[test_index]}")

In [ ]:
pipeline_optimizer.fit(X_train, y_train)